<a href="https://colab.research.google.com/github/vijayshankarrealdeal/GoogleColab/blob/main/TweetsSentiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [3]:
import zipfile
with zipfile.ZipFile('/content/gdrive/MyDrive/Tweet/trainingandtestdata.zip','r') as zip_ref:
  zip_ref.extractall('/content/gdrive/MyDrive/Tweet/')

In [4]:
!pip install tensorflow-gpu
import pandas as pd
import numpy as np
import math
import re
from bs4 import BeautifulSoup
import tensorflow as tf
import tensorflow_datasets as tfds

     |████████████████████████████████| 394.3MB 43kB/s 


In [5]:
cols = ['sentiments','id','date','query','user','text']
dataset_train = pd.read_csv('/content/gdrive/MyDrive/Tweet/training.1600000.processed.noemoticon.csv',header = None,names=cols,engine='python',encoding='latin1')
dataset_test = pd.read_csv("/content/gdrive/MyDrive/Tweet/testdata.manual.2009.06.14.csv",header = None,names=cols,engine='python',encoding='latin1')

In [6]:
dataset_train.drop(['id','date','query','user'],axis = 1,inplace = True)
dataset_train

,sentiments,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,4,Just woke up. Having no school is the best fee...
1599996,4,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...


In [7]:
def clean_tweets(tweets):
    tweets =BeautifulSoup(tweets,"lxml").get_text()
    tweets = re.sub(r"@[a-zA-Z0-9]+",' ',tweets)
    tweets = re.sub(r"https?://[a-zA-Z0-9./]+",' ',tweets)
    tweets = re.sub(r"[^a-zA-Z.!?']",' ',tweets)
    tweets = re.sub(r" +",' ',tweets)
    return tweets


In [8]:
data_labels = dataset_train.sentiments
data_labels[dataset_train.sentiments == 4] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
data_clean = [clean_tweets(tweet) for tweet in dataset_train.text]

In [10]:
data_clean[0]

" Awww that's a bummer. You shoulda got David Carr of Third Day to do it. D"

In [11]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    data_clean,target_vocab_size = 2**16
)
data_input = [tokenizer.encode(sentence) for sentence in data_clean]

In [12]:
MAXLEN = max([len(sentence) for sentence in data_input])
data_inputs = tf.keras.preprocessing.sequence.pad_sequences(data_input,value = 0,padding="post",maxlen=MAXLEN)

In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(data_inputs,data_labels,random_state = 1,test_size = 0.4)

In [14]:
y_train

1346929    1
947578     1
444358     0
752088     0
1438345    1
          ..
836489     1
491263     0
470924     0
491755     0
128037     0
Name: sentiments, Length: 960000, dtype: int64

In [15]:
class CNN(tf.keras.Model):
  def __init__(self,vocab_size,
               emb_dim=128,
               nb_filters = 50,
               dense_units = 512,
               nb_classes = 2,dropout_rating = 0.1,
               Training = False,
               name = "CNN"):
    
    super(CNN,self).__init__(name= name)

    self.embedding = tf.keras.layers.Embedding(vocab_size,emb_dim)

    self.bigram = tf.keras.layers.Conv1D(filters=nb_filters,
                                         kernel_size=2,
                                         padding="valid",
                                         activation = 'relu',
                                         )
    
    self.pool_1 = tf.keras.layers.GlobalMaxPooling1D()

    self.trigram = tf.keras.layers.Conv1D(filters=nb_filters,
                                         kernel_size=3,
                                         padding="valid",
                                         activation = 'relu',
                                         )
    self.pool_2 = tf.keras.layers.GlobalMaxPooling1D()

    self.fourgram = tf.keras.layers.Conv1D(filters=nb_filters,
                                         kernel_size=4,
                                         padding="valid",
                                         activation = 'relu',
                                         )
    self.pool_3 = tf.keras.layers.GlobalMaxPooling1D()

    self.dense_1 = tf.keras.layers.Dense(dense_units,activation = 'relu')
    self.droput = tf.keras.layers.Dropout(dropout_rating)

    if nb_classes == 2:
      self.final_output = tf.keras.layers.Dense(1,activation='sigmoid')
    else:
      self.final_output = tf.keras.layers.Dense(nb_classes,activation='softmax')

    

  def call(self,inputs,training):
    x = self.embedding(inputs)
    x_1 = self.bigram(x)
    x_1 = self.pool_1(x_1)

    x_2 = self.trigram(x)
    x_2 = self.pool_2(x_2)

    x_3 = self.fourgram(x)
    x_3 = self.pool_3(x_3)

    merged = tf.concat([x_1,x_2,x_3],axis = -1)
    merged = self.dense_1(merged)
    merged = self.droput(merged,training)
    output_s = self.final_output(merged)
    return output_s







In [16]:
VOCAB_SIZE = tokenizer.vocab_size
EMB_DIM = 200
NB_FILTERS = 100
units = 256
nb_classes = len(set(y_train))
DROPOUT_RATE = 0.2
BATCH_SIZE = 32
NB_EPOCHS = 200

In [17]:
CNN = CNN(vocab_size=VOCAB_SIZE,
          emb_dim=EMB_DIM,
          nb_filters=NB_FILTERS,
          dense_units=units,
          nb_classes=nb_classes,
          dropout_rating = DROPOUT_RATE,      
          )

In [18]:
if nb_classes == 2:
  CNN.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
else:
  CNN.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['sparse_categorical_crossentropy'])

In [19]:
checkpoint_path = "/content/gdrive/MyDrive/Tweet/chpt"
ckpt = tf.train.Checkpoint(CNN= CNN)
ckpt_manager = tf.train.CheckpointManager(ckpt,checkpoint_path,max_to_keep = 1)
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print("Latest CKPT")



In [20]:
if nb_classes == 2:
  callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy')
else:
  callback = tf.keras.callbacks.EarlyStopping(monitor='sparse_categorical_crossentropy')

In [21]:
CNN.fit(x_train,y_train,batch_size=BATCH_SIZE,epochs=NB_EPOCHS,callbacks=callback)
ckpt.manager.save()

Epoch 1/200
30000/30000 [==============================] - 3386s 112ms/step - loss: 0.4423 - accuracy: 0.7914
Epoch 2/200
30000/30000 [==============================] - 3327s 111ms/step - loss: 0.3267 - accuracy: 0.8596
Epoch 3/200
30000/30000 [==============================] - 3336s 111ms/step - loss: 0.2507 - accuracy: 0.8974
Epoch 4/200
30000/30000 [==============================] - 3338s 111ms/step - loss: 0.1807 - accuracy: 0.9282
Epoch 5/200
11539/30000 [==========>...................] - ETA: 34:19 - loss: 0.1235 - accuracy: 0.9519

KeyboardInterrupt: ignored

In [22]:
CNN.evaluate(x_test,y_test)

20000/20000 [==============================] - 46s 2ms/step - loss: 0.6395 - accuracy: 0.7997


[0.6394509673118591, 0.7996640801429749]